Import Packages and data

In [1]:
import pandas as pd
pd.options.display.max_columns = None
import numpy as np
import os
import regex as re

pop_data = pd.read_csv("../data/population-data2.csv")
housing_prices = pd.read_csv("../data/housing-prices.csv")
business_apps = pd.read_csv('../data/new-business-applications.csv', skiprows=2).dropna(axis=1, how='all')


pop_data = pop_data.query("LSAD == 'Metropolitan Statistical Area'")

In [15]:
business_apps.head()

,State,County,County Code,fipstate,fipscty,BA2005,BA2006,BA2007,BA2008,BA2009,BA2010,BA2011,BA2012,BA2013,BA2014,BA2015,BA2016,BA2017,BA2018,BA2019,BA2020
0,AL,Autauga County,1001,1,1,335,348,348,367,288,304,278,242,282,290,291,294,313,384,393,548
1,AL,Baldwin County,1003,1,3,1986,2101,1975,1721,1505,1548,1553,1594,1532,1575,1735,1858,2114,2104,2311,2675
2,AL,Barbour County,1005,1,5,138,179,156,139,133,107,101,120,121,120,129,122,114,113,146,242
3,AL,Bibb County,1007,1,7,118,145,117,98,98,106,72,95,83,101,78,86,79,105,85,173
4,AL,Blount County,1009,1,9,315,323,297,306,247,247,263,228,225,228,218,254,281,299,300,329


Extract city and state for matching

In [224]:
def split_city(city_list):
    '''
    Takes out city name and other listed cities to isolate main city name fro MSA name
    '''
    return [str.replace(re.split('-|/', s[0])[0], ' County', '') for s in city_list.str.split(',')]

def extract_state(city_list):
    '''
    Extracts main state from list of city names
    '''
    return [str.strip(str.split(s[1], sep='-')[0]) for s in city_list.str.split(',')]

pop_data.insert(5, 'city_stripped', split_city(pop_data.NAME))
pop_data.insert(6, 'STATE', extract_state(pop_data.NAME))
housing_prices.insert(3, 'city_stripped', split_city(housing_prices.RegionName))

#### Population Data

separate out by series type and strip column titles

In [225]:
pop_base_cols = ['CBSA', 'city_stripped', 'STATE']
estimate_cols = list(pop_data.columns[pop_data.columns.str.find('POPESTIMATE') > -1])
pop_change_cols = list(pop_data.columns[pop_data.columns.str.find('NPOPCHG') > -1])
birth_cols = list(pop_data.columns[pop_data.columns.str.find('BIRTHS') > -1])
death_cols = list(pop_data.columns[pop_data.columns.str.find('DEATHS') > -1])
natinc_cols = list(pop_data.columns[pop_data.columns.str.find('NATURALINC') > -1])
int_mig_cols = list(pop_data.columns[pop_data.columns.str.find('INTERNATIONAL') > -1])
dom_mig_cols = list(pop_data.columns[pop_data.columns.str.find('DOMESTIC') > -1])
net_mig_cols = list(pop_data.columns[pop_data.columns.str.find('NETMIG') > -1])

In [226]:
pop_pop = pop_data[pop_base_cols + estimate_cols]
pop_pop.columns = pop_pop.columns.str.replace('POPESTIMATE', '')

pop_change = pop_data[pop_base_cols + pop_change_cols]
pop_change.columns = pop_change.columns.str.replace('NPOPCHG', '')

pop_birth = pop_data[pop_base_cols + birth_cols]
pop_birth.columns = pop_births.columns.str.replace('BIRTHS', '')

pop_death = pop_data[pop_base_cols + death_cols]
pop_death.columns = pop_death.columns.str.replace('DEATHS', '')

pop_natinc = pop_data[pop_base_cols + natinc_cols]
pop_natinc.columns = pop_natinc.columns.str.replace('NATURALINC', '')

pop_intmig = pop_data[pop_base_cols + int_mig_cols]
pop_intmig.columns = pop_intmig.columns.str.replace('INTERNATIONALMIG', '')

pop_dommig = pop_data[pop_base_cols + dom_mig_cols]
pop_dommig.columns = pop_dommig.columns.str.replace('DOMESTICMIG', '')

pop_netmig = pop_data[pop_base_cols + net_mig_cols]
pop_netmig.columns = pop_netmig.columns.str.replace('NETMIG', '')

melt individual dfs and merge together

In [227]:
pop_pop = pd.melt(pop_pop, id_vars=pop_base_cols).rename(columns={'variable' : 'year', 'value' : 'population'})

pop_change = pd.melt(pop_change, id_vars=pop_base_cols).rename(columns={'variable' : 'year', 'value' : 'population_change'})

pop_birth = pd.melt(pop_birth, id_vars=pop_base_cols).rename(columns={'variable' : 'year', 'value' : 'births'})

pop_death = pd.melt(pop_death, id_vars=pop_base_cols).rename(columns={'variable' : 'year', 'value' : 'deaths'})

pop_natinc = pd.melt(pop_natinc, id_vars=pop_base_cols).rename(columns={'variable' : 'year', 'value' : 'nat_increase'})

pop_intmig = pd.melt(pop_intmig, id_vars=pop_base_cols).rename(columns={'variable' : 'year', 'value' : 'int_migration'})

pop_dommig = pd.melt(pop_dommig, id_vars=pop_base_cols).rename(columns={'variable' : 'year', 'value' : 'dom_migration'})

pop_netmig = pd.melt(pop_netmig, id_vars=pop_base_cols).rename(columns={'variable' : 'year', 'value' : 'net_migration'})

pop_dfs = [pop_change, pop_birth, pop_death, pop_natinc, pop_intmig, pop_dommig, pop_netmig]

In [228]:
pop_cleaned = pop_pop

for df in pop_dfs:
    pop_cleaned = pd.merge(pop_cleaned, df, how='left', on=pop_base_cols + ['year'], validate='one_to_one')

In [229]:
pop_cleaned.head()

,CBSA,city_stripped,STATE,year,population,population_change,births,deaths,nat_increase,int_migration,dom_migration,net_migration
0,10180,Abilene,TX,2010,165585,333,539,409,130,84,124,208
1,10420,Akron,OH,2010,703031,-165,1980,1606,374,221,-727,-506
2,10500,Albany,GA,2010,154145,112,540,409,131,18,-40,-22
3,10540,Albany,OR,2010,116891,210,386,260,126,20,74,94
4,10580,Albany,NY,2010,871082,369,2379,1796,583,442,-618,-176


#### Housing Data

In [240]:
housing_cleaned = pd.melt(housing_prices.drop(columns=['RegionID', 'SizeRank', 'RegionName', 'RegionType'])
, id_vars = ['city_stripped', 'StateName']).dropna(subset=['StateName']).rename(columns={'variable' : 'time'})

In [246]:
housing_cleaned['year'] = [h[0] for h in housing_cleaned['time'].str.split('-')]
housing_cleaned['month'] = [h[1] for h in housing_cleaned['time'].str.split('-')]

housing_cleaned.head()

,city_stripped,StateName,time,value,year,month
1,New York,NY,2000-01-31,224341.0,2000,01
2,Los Angeles,CA,2000-01-31,231193.0,2000,01
3,Chicago,IL,2000-01-31,170104.0,2000,01
4,Dallas,TX,2000-01-31,130035.0,2000,01
5,Philadelphia,PA,2000-01-31,129808.0,2000,01


#### Merge

In [252]:
pop_house = pd.merge(housing_cleaned.drop(columns=['time']), pop_cleaned,
                     how='inner', left_on=['city_stripped', 'StateName', 'year'],
                     right_on= ['city_stripped', 'STATE', 'year'],
                     indicator='matched', validate='many_to_one')

In [267]:
pop_house.head()

,city_stripped,StateName,value,year,month,CBSA,STATE,population,population_change,births,deaths,nat_increase,int_migration,dom_migration,net_migration,matched
0,New York,NY,402418.0,2010,01,35620,NY,18923407,27130,61880,31929,29951,21771,-24080,-2309,both
1,New York,NY,402709.0,2010,02,35620,NY,18923407,27130,61880,31929,29951,21771,-24080,-2309,both
2,New York,NY,402528.0,2010,03,35620,NY,18923407,27130,61880,31929,29951,21771,-24080,-2309,both
3,New York,NY,402482.0,2010,04,35620,NY,18923407,27130,61880,31929,29951,21771,-24080,-2309,both
4,New York,NY,402177.0,2010,05,35620,NY,18923407,27130,61880,31929,29951,21771,-24080,-2309,both


In [266]:
pop_house.value.isna().sum()

368

*Some null values for home value but not many, every city has 120 observations (full match for 120 months of data*

Next - pull in business apps, taxes and unemployment

In [200]:
pd.merge(pop_pop, pop_cleaned, how='outer', on=pop_base_cols + ['year'])

,CBSA,city_stripped,STATE,year,population_x,population_y,population_change,births,deaths,nat_increase,int_migration,dom_migration,net_migration
0,10180,Abilene,TX,2010,165585,165585,NaN,539,409,130,NaN,NaN,208
1,10420,Akron,OH,2010,703031,703031,NaN,1980,1606,374,NaN,NaN,-506
2,10500,Albany,GA,2010,154145,154145,NaN,540,409,131,NaN,NaN,-22
3,10540,Albany,OR,2010,116891,116891,NaN,386,260,126,NaN,NaN,94
4,10580,Albany,NY,2010,871082,871082,NaN,2379,1796,583,NaN,NaN,-176
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3835,49420,Yakima,WA,2019,250873,250873,NaN,3746,2093,1653,NaN,NaN,-1340
3836,49620,York,PA,2019,449058,449058,NaN,4868,4121,747,NaN,NaN,491
3837,49660,Youngstown,OH,2019,536081,536081,NaN,5450,6842,-1392,NaN,NaN,-726
3838,49700,Yuba City,CA,2019,175639,175639,NaN,2416,1417,999,NaN,NaN,737


In [129]:
pop_house = pd.merge(housing_prices.iloc[:, 0:10], pop_data.iloc[:, 0:10],
                     how='left', left_on=['city_stripped', 'StateName'],
                     right_on= ['city_stripped', 'STATE'],
                     indicator='matched', validate='one_to_one')

In [122]:
housin.shape

NameError: name 'housin' is not defined

In [130]:
pop_house.query("matched=='left_only'")

,RegionID,SizeRank,RegionName,city_stripped,RegionType,StateName,2000-01-31,2000-02-29,2000-03-31,2000-04-30,CBSA,MDIV,STCOU,NAME,LSAD,STATE,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,matched
0,102001.0,0.0,United States,United States,Country,NaN,127202.0,127546.0,127908.0,128645.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
58,394415.0,58.0,"Stamford, CT",Stamford,Msa,CT,278378.0,280700.0,282070.0,284934.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
66,394952.0,66.0,"Ventura, CA",Ventura,Msa,CA,263606.0,264558.0,265834.0,268185.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
84,394440.0,84.0,"Fort Myers, FL",Fort Myers,Msa,FL,139513.0,139768.0,139986.0,140539.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
90,394528.0,90.0,"Daytona Beach, FL",Daytona Beach,Msa,FL,107868.0,108013.0,108428.0,109068.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
903,394767.0,929.0,"Lamesa, TX",Lamesa,Msa,TX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
904,753874.0,930.0,"Craig, CO",Craig,Msa,CO,94741.0,95085.0,95668.0,96360.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
905,394968.0,931.0,"Pecos, TX",Pecos,Msa,TX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
906,395188.0,932.0,"Vernon, TX",Vernon,Msa,TX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only


In [104]:
pop_data.loc[pop_data['NAME'].str.contains('Austin')][['NAME', 'POPESTIMATE2019', 'city_stripped']]

,NAME,POPESTIMATE2019,city_stripped
106,"Austin-Round Rock-Georgetown, TX",2227083,Austin
645,"Austin County, TX",30032,Austin
1664,"Austin, MN",40062,Austin


In [94]:
housing_prices.RegionName[:50]

0                          United States
1                           New York, NY
2     Los Angeles-Long Beach-Anaheim, CA
3                            Chicago, IL
4                  Dallas-Fort Worth, TX
5                       Philadelphia, PA
6                            Houston, TX
7                         Washington, DC
8              Miami-Fort Lauderdale, FL
9                            Atlanta, GA
10                            Boston, MA
11                     San Francisco, CA
12                           Detroit, MI
13                         Riverside, CA
14                           Phoenix, AZ
15                           Seattle, WA
16               Minneapolis-St Paul, MN
17                         San Diego, CA
18                         St. Louis, MO
19                             Tampa, FL
20                         Baltimore, MD
21                            Denver, CO
22                        Pittsburgh, PA
23                          Portland, OR
24              